<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/cartinaMondo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

# Percorso del file
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetCartinaMondo.csv")

def generate_world_smoking_map():
  # Controllo per il nome del path
    if not os.path.exists(file_path):
        print(f"Errore: non c'è '{file_path}'.")
        return

    try:
        # Caricamento della tabella
        df = pd.read_csv(file_path)

        # Filtraggio basato sulla struttura della tabella
        # Anno 2025 e il totale della popolazione
        mask = (
            (df['DIM_TIME'] == 2025) &
            (df['DIM_SEX'] == 'TOTAL')
        )
        df_filtered = df[mask].copy()

        # Pulizia dati con solo valori siano numerici
        df_filtered['PERCENT_POP_N'] = pd.to_numeric(df_filtered['PERCENT_POP_N'], errors='coerce')
        df_filtered = df_filtered.dropna(subset=['PERCENT_POP_N', 'GEO_NAME_SHORT'])

        # Barra dei colori a lato
        real_min = df_filtered['PERCENT_POP_N'].min()
        real_max = df_filtered['PERCENT_POP_N'].max()
        # Valore dummy per la sezione grigia
        dummy_val = real_min - 5

        # Riga "finta" per la sezione grigia
        dummy_row = pd.DataFrame({
            'GEO_NAME_SHORT': ['Dati non pervenuti'],
            'PERCENT_POP_N': [dummy_val]
        })
        df_plot = pd.concat([df_filtered, dummy_row], ignore_index=True)

        # Scala colori grigio poi i blu
        custom_blues = px.colors.sequential.Blues
        custom_scale = [
            [0.0, "#cccccc"],           # Sezione grigia
            [0.1, "#cccccc"],
            [0.1, custom_blues[0]],    # Inizio dei blu
            [1.0, custom_blues[-1]]     # Massimo dei blu
        ]

        # 3. Creazione Mappa Mondiale
        fig = px.choropleth(
            df_plot,
            locations="GEO_NAME_SHORT",
            locationmode='country names', # Uso i nomi degli stati presenti nel CSV
            color="PERCENT_POP_N",
            hover_name="GEO_NAME_SHORT",
            color_continuous_scale=custom_scale,
            range_color=[dummy_val, real_max],
            scope="world",
            labels={'PERCENT_POP_N': '% Fumatori'}
        )

        # 4. Configurazione Mppa
        fig.update_geos(
            showframe=False,
            showcoastlines=True,
            showcountries=True,
            countrycolor="#444",       # Colore confini
            landcolor="#cccccc",       # Sfondo per paesi senza dati
            projection_type="natural earth",
            resolution=50,
            showlakes=False # Toglie i laghi
        )

        # 5. Titolo e sottotitolo
        fig.update_layout(
            title={
                'text': '<span style="color:#2E4053; font-size:24px; font-weight:bold;">Densità Fumatori nel Mondo</span><br><span style="color:#95A5A6; font-size:16px;">Dati stimati per il 2025 • Percentuale popolazione</span>',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            margin={"r":0,"t":100,"l":0,"b":0},
            showlegend=False,
            coloraxis_colorbar=dict(
                title=dict(
                    text="% Popolazione<br> &nbsp;", # Titolo barra colori
                    side="top"
                ),
                thickness=20,
                len=0.6,
                x=0.92,
                y=0.5,
                tickvals=[dummy_val, real_min, (real_min + real_max)/2, real_max],
                ticktext=["Dati non sufficienti", f"{real_min:.1f}%", f"{(real_min + real_max)/2:.1f}%", f"{real_max:.1f}%"],
                tickmode="array"
            )
        )

        # 6. Pulizia Hover
        fig.update_traces(
            hovertemplate="<b>%{hovertext}</b><br>Prevalenza: %{z}%<extra></extra>"
        )

        fig.show()

    except Exception as e:
        print(f"Errore durante l'elaborazione della mappa mondiale: {e}")

if __name__ == "__main__":
    generate_world_smoking_map()